# Adapter Assignment Extractor (step == 0)

이 노트북은 `/mnt/data/train_results.raw`를 **line-by-line**으로 읽어서,

1) `client -> adapter_idx` 매핑을 만들고,
2) **`adapter_idx -> [client indices]`** 매핑도 함께 생성/저장합니다.

In [4]:
import json
from collections import OrderedDict, defaultdict
from typing import Any, Dict, List

RAW_PATH = "./train_results.raw"  # 필요 시 변경

def safe_int(x):
    try:
        return int(x)
    except Exception:
        return None

def find_key_by_pred(d: dict, pred):
    for k in d.keys():
        try:
            if pred(k):
                return k
        except Exception:
            pass
    return None

def get_step(rec: Dict[str, Any]):
    if "step" in rec:
        return safe_int(rec["step"])
    k = find_key_by_pred(rec, lambda x: "step" in x.lower())
    return safe_int(rec.get(k)) if k else None

def get_client(rec: Dict[str, Any]):
    if "client" in rec:
        return safe_int(rec["client"])
    k = find_key_by_pred(rec, lambda x: "client" in x.lower() or "cid" in x.lower())
    return safe_int(rec.get(k)) if k else None

def get_adapter_idx(rec: Dict[str, Any]):
    for key in ("adapter_idx", "Adapter", "adapter"):
        if key in rec:
            return safe_int(rec[key])
    k = find_key_by_pred(rec, lambda x: "adapter" in x.lower() or x.lower().endswith("ter_idx"))
    if k is not None:
        return safe_int(rec.get(k))
    for v in rec.values():
        if isinstance(v, dict):
            val = get_adapter_idx(v)
            if val is not None:
                return val
    return None

def build_adapter_map(raw_path: str, num_clients: int = 53) -> OrderedDict:
    mapping: Dict[int, int] = {}
    with open(raw_path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            try:
                rec = json.loads(s)
            except Exception:
                continue
            if get_step(rec) != 0:
                continue
            cid = get_client(rec)
            ai  = get_adapter_idx(rec)
            if isinstance(cid, int) and 1 <= cid <= num_clients and cid not in mapping and isinstance(ai, int):
                mapping[cid] = ai
    return OrderedDict(sorted(mapping.items(), key=lambda kv: kv[0]))

client_to_adapter = build_adapter_map(RAW_PATH, num_clients=53)
len(client_to_adapter), list(client_to_adapter.items())[:10]

(53,
 [(1, 4),
  (2, 6),
  (3, 2),
  (4, 1),
  (5, 2),
  (6, 1),
  (7, 3),
  (8, 0),
  (9, 0),
  (10, 0)])

## adapter_idx -> [client indices] 매핑 생성

In [5]:
from collections import defaultdict
adapter_to_clients: Dict[int, List[int]] = defaultdict(list)
for cid, aid in client_to_adapter.items():
    adapter_to_clients[aid].append(cid)



# 정렬 정리: 키 정렬, value 리스트도 정렬
adapter_to_clients = OrderedDict(sorted((k, sorted(v)) for k, v in adapter_to_clients.items()))
dict(adapter_to_clients)

{0: [8,
  9,
  10,
  11,
  14,
  15,
  16,
  17,
  18,
  19,
  23,
  26,
  27,
  32,
  34,
  35,
  38,
  41,
  44,
  45,
  52,
  53],
 1: [4, 6, 13, 25, 31, 33],
 2: [3, 5],
 3: [7, 46],
 4: [1, 22, 28, 30, 36, 37, 43, 47, 48, 50, 51],
 5: [21, 24, 29, 39],
 6: [2, 12, 20, 40, 42, 49]}

## 결과 미리보기 및 저장 (JSON/CSV)

In [6]:
import pandas as pd
from IPython.display import display

# (1) client -> adapter 미리보기/저장
df_client_to_adapter = pd.DataFrame([{"client": k, "adapter_idx": v} for k, v in client_to_adapter.items()])
display(df_client_to_adapter.head(15))



# (2) adapter -> [clients] 미리보기/저장
df_adapter_to_clients = pd.DataFrame([
    {"adapter_idx": k, "clients": ",".join(map(str, v)), "count": len(v)}
    for k, v in adapter_to_clients.items()
])
display(df_adapter_to_clients)



,client,adapter_idx
0,1,4
1,2,6
2,3,2
3,4,1
4,5,2
5,6,1
6,7,3
7,8,0
8,9,0
9,10,0


,adapter_idx,clients,count
0,0,"8,9,10,11,14,15,16,17,18,19,23,26,27,32,34,35,...",22
1,1,"4,6,13,25,31,33",6
2,2,"3,5",2
3,3,"7,46",2
4,4,"1,22,28,30,36,37,43,47,48,50,51",11
5,5,"21,24,29,39",4
6,6,"2,12,20,40,42,49",6
